In [ ]:
# =========================== #
#       Import Libraries      #
# =========================== #
import os
import cv2
import random
import shutil
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import imghdr

from keras.applications import MobileNetV2, VGG16, DenseNet121
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# =========================== #
#     Image Size Settings     #
# =========================== #
img_height, img_width = 224, 224

# =========================== #
#     Data Generator Setup    #
# =========================== #
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Aug_data2/train',
                                                    target_size=(img_height, img_width),
                                                    batch_size=16,
                                                    class_mode='binary')

val_generator = val_datagen.flow_from_directory('/content/drive/MyDrive/Aug_data2/val',
                                                target_size=(img_height, img_width),
                                                batch_size=16,
                                                class_mode='binary')

test_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/Aug_data2/test',
                                                  target_size=(img_height, img_width),
                                                  batch_size=32,
                                                  class_mode='binary',
                                                  shuffle=False)

# =========================== #
#     1. MobileNetV2 Model    #
# =========================== #
mobilenet_v2 = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

for layer in mobilenet_v2.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(mobilenet_v2.output)
x = Dropout(0.5)(x)
prediction = Dense(1, activation='sigmoid')(x)
model_mobilenet = Model(inputs=mobilenet_v2.input, outputs=prediction)

model_mobilenet.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

history_mobilenet = model_mobilenet.fit(train_generator,
                                        validation_data=val_generator,
                                        epochs=20,
                                        verbose=2)

# Plot MobileNetV2 Accuracy & Loss
plt.plot(history_mobilenet.history['accuracy'], color='red', label='train')
plt.plot(history_mobilenet.history['val_accuracy'], color='blue', label='validation')
plt.legend()
plt.title('MobileNetV2 Accuracy')
plt.show()

plt.plot(history_mobilenet.history['loss'], color='red', label='train')
plt.plot(history_mobilenet.history['val_loss'], color='blue', label='validation')
plt.title('MobileNetV2 Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

test_loss, test_accuracy = model_mobilenet.evaluate(test_generator)
print(f"MobileNetV2 Test Loss: {test_loss}")
print(f"MobileNetV2 Test Accuracy: {test_accuracy}")

model_mobilenet.save('/content/drive/MyDrive/Aug_data_bmp/total_data/train_test_val/without_seg/mobileNetV2_without_seg_data_model.h5')
print("MobileNetV2 model saved successfully.")

# =========================== #
#         2. VGG16 Model      #
# =========================== #
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

for layer in vgg16.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(vgg16.output)
x = Dropout(0.5)(x)
prediction = Dense(1, activation='sigmoid')(x)
model_vgg16 = Model(inputs=vgg16.input, outputs=prediction)

model_vgg16.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

history_vgg16 = model_vgg16.fit(train_generator,
                                validation_data=val_generator,
                                epochs=20,
                                verbose=2)

# Plot VGG16 Accuracy & Loss
plt.plot(history_vgg16.history['accuracy'], color='red', label='train')
plt.plot(history_vgg16.history['val_accuracy'], color='blue', label='validation')
plt.legend()
plt.title('VGG16 Accuracy')
plt.show()

plt.plot(history_vgg16.history['loss'], color='red', label='train')
plt.plot(history_vgg16.history['val_loss'], color='blue', label='validation')
plt.title('VGG16 Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

test_loss, test_accuracy = model_vgg16.evaluate(test_generator)
print(f"VGG16 Test Loss: {test_loss}")
print(f"VGG16 Test Accuracy: {test_accuracy}")

model_vgg16.save('/content/drive/MyDrive/Aug_data_bmp/total_data/train_test_val/without_seg/vgg16_without_seg_data_model.h5')
print("VGG16 model saved successfully.")

# =========================== #
#       3. DenseNet121 Model  #
# =========================== #
densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

for layer in densenet.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(densenet.output)
x = Dropout(0.5)(x)
prediction = Dense(1, activation='sigmoid')(x)
model_densenet = Model(inputs=densenet.input, outputs=prediction)

model_densenet.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

history_densenet = model_densenet.fit(train_generator,
                                      validation_data=val_generator,
                                      epochs=20,
                                      verbose=2)

# Plot DenseNet121 Accuracy & Loss
plt.plot(history_densenet.history['accuracy'], color='red', label='train')
plt.plot(history_densenet.history['val_accuracy'], color='blue', label='validation')
plt.legend()
plt.title('DenseNet121 Accuracy')
plt.show()

plt.plot(history_densenet.history['loss'], color='red', label='train')
plt.plot(history_densenet.history['val_loss'], color='blue', label='validation')
plt.title('DenseNet121 Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

test_loss, test_accuracy = model_densenet.evaluate(test_generator)
print(f"DenseNet121 Test Loss: {test_loss}")
print(f"DenseNet121 Test Accuracy: {test_accuracy}")

model_densenet.save('/content/drive/MyDrive/Aug_data_bmp/total_data/train_test_val/without_seg/densenet_without_seg_data_model.h5')
print("DenseNet121 model saved successfully.")